<a href="https://colab.research.google.com/github/jpsbali/GoogleColab/blob/main/Apple_CLaRa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torch
%pip install -U transformers==4.57.1 trl==0.25.1 datasets==4.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
from google.colab import userdata
from huggingface_hub import login

# Login into Hugging Face Hub
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [3]:
from huggingface_hub import snapshot_download
import os

# Define the model repo ID and the specific subfolder you want
REPO_ID = "apple/CLaRa-7B-Instruct"
SUBFOLDER_NAME = "compression-16"

# Define the local directory where you want to save the files
LOCAL_DIR = "./local_model_folder"

# Create the local directory if it doesn't exist
os.makedirs(LOCAL_DIR, exist_ok=True)

# Download only the files within the specified subfolder
# The 'allow_patterns' argument takes a list of patterns to match files/folders
downloaded_folder_path = snapshot_download(
    repo_id=REPO_ID,
    local_dir=LOCAL_DIR,
    allow_patterns=[f"{SUBFOLDER_NAME}/*"],
    ignore_patterns=["*.safetensors", "*.bin"] # Optional: ignore large files if not needed
)

print(f"Downloaded subfolder located at: {downloaded_folder_path}/{SUBFOLDER_NAME}")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

compression-16/adapters.pth:   0%|          | 0.00/693M [00:00<?, ?B/s]

compression-16/decoder_first_last_layers(…):   0%|          | 0.00/525M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

results_metrics_stage1_True.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

modeling_clara.py: 0.00B [00:00, ?B/s]

compression-16/tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Downloaded subfolder located at: /content/local_model_folder/compression-16


In [5]:
from transformers import AutoModel, AutoTokenizer
import torch
import os

#Construct the full local path to the 'compression-16' model files
local_model_path = os.path.join('/content/local_model_folder', 'compression-16')

#Load the model and tokenizer from the local path
unirag = AutoModel.from_pretrained(
    local_model_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    local_files_only=True).to("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained( local_model_path, trust_remote_code=True, local_files_only=True )

documents = [ [
    "Weldenia is a monotypic genus of flowering plant in the family Commelinaceae...",
    "Hagsatera is a genus of flowering plants from the orchid family...",
    "Alsobia is a genus of flowering plants in the family Gesneriaceae..." ] ]

questions = [ "Which genus of plant grows originally in Mexico and Guatemala, Phylica or Weldenia?" ]

#Instruction-tuned usage
out = unirag.generate_from_text( questions=questions, documents=documents, max_new_tokens=64 )

print("Generated answer:", out)

<unknown>:997: SyntaxWarning: invalid escape sequence '\ '
<unknown>:1040: SyntaxWarning: invalid escape sequence '\ '
<unknown>:1062: SyntaxWarning: invalid escape sequence '\ '
<unknown>:1085: SyntaxWarning: invalid escape sequence '\ '
<unknown>:1107: SyntaxWarning: invalid escape sequence '\ '
<unknown>:1133: SyntaxWarning: invalid escape sequence '\ '
<unknown>:1157: SyntaxWarning: invalid escape sequence '\ '
/root/.cache/huggingface/modules/transformers_modules/compression_hyphen_16/modeling_clara.py:997: SyntaxWarning: invalid escape sequence '\ '
  user_prompt = [{"role": "user", "content": prompt_user.replace(':\ ', ': ')}]
/root/.cache/huggingface/modules/transformers_modules/compression_hyphen_16/modeling_clara.py:1040: SyntaxWarning: invalid escape sequence '\ '
  user_prompt = [{"role": "user", "content": prompt_user.replace(':\ ', ': ')}]
/root/.cache/huggingface/modules/transformers_modules/compression_hyphen_16/modeling_clara.py:1062: SyntaxWarning: invalid escape sequ

Initializing model from trained checkpoint: CLaRaConfig {
  "_attn_implementation_autoset": true,
  "ae_mode": "token",
  "attn_implementation": null,
  "auto_map": {
    "AutoConfig": "modeling_clara.CLaRaConfig",
    "AutoModel": "modeling_clara.CLaRa"
  },
  "compr_base_model_name": "mistralai/Mistral-7B-Instruct-v0.2",
  "compr_every_n_layer": null,
  "compr_linear_type": "concat",
  "compr_mlp_hidden_dim": 8096,
  "compr_model_name": null,
  "compr_n_layers": 5,
  "compr_rate": 16,
  "compr_rms_norm": false,
  "compr_use_mlp": false,
  "decoder_model_name": "mistralai/Mistral-7B-Instruct-v0.2",
  "device_map": null,
  "different_mem_tokens": true,
  "doc_max_length": 256,
  "dtype": "bfloat16",
  "generation_top_k": 5,
  "kbtc_training": false,
  "load_adapters": false,
  "load_pretrained_checkpoint": false,
  "lora": true,
  "lora_compressor": false,
  "lora_r": 16,
  "lora_r_compressor": 16,
  "max_new_tokens": 128,
  "model_type": "CLaRa",
  "optimize_mem_tokens": true,
  "pad_

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

Memory token count: 16


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Adapter decoder_adapter trainable parameters: 41943040
Loading checkpoint adapter: encoder_adapter


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 4979 has 14.73 GiB memory in use. Of the allocated memory 14.49 GiB is allocated by PyTorch, and 117.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)